In [1]:
import easyocr
import cv2
import numpy as np
from PIL import Image
reader = easyocr.Reader(['de'])
file = 'straighted_orin.jpg'
prediction = np.array(reader.readtext(file))


In [272]:
# horizontal_list, free_list = reader.custom_text_detection(file)
# prediction = horizontal_list
#prediction = np.array(reader.custom_text_recognition(file, horizontal_list, free_list))

In [2]:
SHOW_IMAGE = True
# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
fontScale = 0.75
# Blue color in BGR 
color = (255, 0, 0) 
# Line thickness of 2 px 
thickness = 2

def show(np_img, name="image", show=SHOW_IMAGE):
    if show:
        Image.fromarray(np_img).show(name)

def get_hight_of_box(box):
    return (box[2][1]+box[3][1]-box[0][1]-box[1][1])/2

def box_filter(prediction, image_size):
    prediction = np.array(prediction)
    hights = []
    box_index = np.zeros(len(prediction)) 
    for box_out in prediction:
        box = box_out[0]
        h = get_hight_of_box(box)
        hights.append(int(h))
    hights = np.array(hights)
    prediction_out =  prediction[hights>0.75*np.mean(hights)]
    return prediction_out

def draw_image(image_c, prediction, texts = None, is_show = False):
    for i in range(prediction.shape[0]):
        box_out = prediction[i]
        box = box_out[0]
        points = np.reshape(box, [4, 2]).astype(np.int32)
        cv2.line(image_c,tuple(points[0]),tuple(points[1]),(0,0,255),2)
        cv2.line(image_c,tuple(points[0]),tuple(points[3]),(0,0,255),2)
        cv2.line(image_c,tuple(points[1]),tuple(points[2]),(0,0,255),2)
        cv2.line(image_c,tuple(points[2]),tuple(points[3]),(0,0,255),2)
        if texts is None:
            image_c = cv2.putText(image_c, str(box_out[1]), ((points[0][0], points[0][1])), font, fontScale, color, thickness, cv2.LINE_AA)
        else:
            image_c = cv2.putText(image_c, str(texts[i]), ((points[0][0], points[0][1])), font, fontScale, color, thickness, cv2.LINE_AA)
    if is_show:
        show(image_c)
    return image_c

def draw_box(image_c, box, color = (255,0,255)):
    cv2.line(image_c,tuple(box[0]),tuple(box[1]),color,2)
    cv2.line(image_c,tuple(box[0]),tuple(box[3]),color,2)
    cv2.line(image_c,tuple(box[1]),tuple(box[2]),color,2)
    cv2.line(image_c,tuple(box[2]),tuple(box[3]),color,2)
    return image_c

def sort_by_hpos(boxes):
    pos = np.array(boxes[:, 0])
    hpos = []
    for p in pos:
        hpos.append((p[0][1]+p[2][1])//2)
    hpos = np.array(hpos)
    boxes = boxes[hpos.argsort()]
    hpos = np.sort(hpos)
    return hpos, boxes

def sort_by_wpos(boxes):
    pos = np.array(boxes[:, 0])
    wpos = []
    for p in pos:
        wpos.append((p[0][0]+p[2][0])//2)
    wpos = np.array(wpos)
    boxes = boxes[wpos.argsort()]
    wpos = np.sort(wpos)
    return wpos, boxes

def sort_boxes_by_line(boxes):
    max_dis_rate = 2.5
    hpos, boxes = sort_by_hpos(boxes)
    boxes_out = []
    hc = get_hight_of_box(boxes[0][0])
    max_dis = hc//max_dis_rate
    boxes_out_row_i = []
    line_hpos = []
    pc = hpos[0]
    for i in range(boxes.shape[0]):
        if abs(hpos[i]- pc)<max_dis:
            boxes_out_row_i.append(boxes[i])
        else:
            wpos, sorted_row = sort_by_wpos(np.array(boxes_out_row_i))
            boxes_out.append(sorted_row)
            boxes_out_row_i = [boxes[i]]
            hc = get_hight_of_box(boxes[i][0])
            max_dis = hc//max_dis_rate
            pc = hpos[i]
            
        if i == boxes.shape[0]-1:
            wpos, sorted_row = sort_by_wpos(np.array(boxes_out_row_i))
            boxes_out.append(sorted_row)
    return boxes_out 

def line_hpos_caculator(line):
    h_min = line[0][0][0][1]
    h_max = 0
    for box in line:
        for point in box[0]:
            if h_min > point[1]:
                h_min = point[1]
            if h_max< point[1]:
                h_max = point[1]
    return (h_min+h_max)//2

def box_merge(boxes):
    h_min = boxes[0][0][0][1]
    h_max = 0
    w_min = boxes[0][0][0][0]
    w_max = 0
    for box in boxes:
        for point in box[0]:
            if h_min > point[1]:
                h_min = point[1]
            if h_max< point[1]:
                h_max = point[1]
            if w_min > point[0]:
                w_min = point[0]
            if w_max< point[0]:
                w_max = point[0]
    return [[w_min, h_min], [w_max, h_min], [w_max, h_max], [w_min, h_max]]


In [3]:
left_side_config=[
    {"num":1, "hpos":0.134},
    {"num":1, "hpos":0.45},
    {"num":1, "hpos":0.517},
    {"num":1, "hpos":0.705},
    {"num":1, "hpos":-1},
    {"num":2, "hpos":0.893, "march":[0.5]},
]

In [6]:
image_c = cv2.imread(file)
image_c = cv2.cvtColor(image_c, cv2.COLOR_BGR2RGB)
image_r = image_c.copy()
h, w = image_c.shape[:2]
prediction_filtered = box_filter(prediction, (h, w))
#print(prediction_filtered[0])
left_bound = int(w/3.6)
right_bound = int(w/1.6)
draw_image(image_c.copy(), prediction, is_show = True)
draw_image(image_c.copy(), prediction_filtered, is_show = True)
boxes_xpos = np.zeros(prediction_filtered.shape[0])

cv2.line(image_c, (left_bound, 0),(left_bound, h), (0,255,0),2)
cv2.line(image_c, (right_bound, 0),(right_bound, h), (0,255,0),2)
for i in range(prediction_filtered.shape[0]):
    boxes_xpos[i]=prediction_filtered[i][0][0][0]

########### LEFT AREA
left_boxes = prediction_filtered[boxes_xpos<left_bound]
left_boxes_sorted = sort_boxes_by_line(left_boxes)
print(len(left_boxes_sorted))
draw_image(image_c.copy(), left_boxes, is_show = True)
left_boxes_hpos = []
for i in range(len(left_boxes_sorted)):
    texts = []
    stt = 0
    for box in left_boxes_sorted[i]:
        stt+=1
        texts.append(str(i+1)+"*"+str(stt))
    hpos = line_hpos_caculator(left_boxes_sorted[i])
    left_boxes_hpos.append(hpos)
    image_c = draw_image(image_c, left_boxes_sorted[i], texts)
print(np.array(left_boxes_hpos)/h) 

show(image_c) 

def line_process(line, line_stt, image_r):
    boxes = left_boxes_sorted[line_stt]
    if line.get("num")==1:
        box_merged = box_merge(boxes)
        image_r = draw_box(image_r, box_merged)
    else:
        m = 0
        box_group = []
        for j in range(len(boxes)):
            if boxes[j][0][0][0]<line.get("march")[m]*left_bound:
                box_group.append(boxes[j])
            else:
                if len(box_group)>0:
                    box_merged = box_merge(box_group)
                    image_r = draw_box(image_r, box_merged, (255,0,0))
                    box_group= []
                    m = m+1
                    if m ==line.get("num")-1:
                        box_merged = box_merge(boxes[j:])
                        image_r = draw_box(image_r, box_merged, (255,0,0))
                        break
    return image_r

last_line = 0   
for line in left_side_config:
    hpos = line.get("hpos")*h
    if hpos<0:
        image_r = line_process(line, last_line+1, image_r)
        last_line+=1
    else:
        dis = abs(left_boxes_hpos[0]-hpos)
        for i in range(len(left_boxes_hpos)):
            this_dis = abs(left_boxes_hpos[i]-hpos)
            if this_dis>dis or i == len(left_boxes_hpos)-1:
                if i == len(left_boxes_hpos)-1 and  this_dis<=dis:
                    i+=1
                image_r = line_process(line, i-1, image_r)
                last_line = i-1
                break
            else:
                dis = this_dis
show(image_r)
########### MIDDLE AREA
# middle_boxes = prediction_filtered[boxes_xpos>right_bound]
# middle_boxes_sorted = sort_boxes_by_line(middle_boxes)

# for i in range(len(middle_boxes_sorted)):
#     texts = []
#     stt = 0
#     for box in middle_boxes_sorted[i]:
#         stt+=1
#         texts.append(str(i+1)+"-"+str(stt))
#     image_c = draw_image(image_c, middle_boxes_sorted[i], texts)       

12
[0.05882353 0.09064609 0.14079074 0.17936355 0.19382835 0.25458052
 0.45998071 0.5265188  0.71745419 0.75891996 0.90260366 0.93249759]
